In [1]:
import os
import ctypes
import numpy as np
import pandas as pd

from numpy.ctypeslib import ndpointer

In [2]:
!cc -fPIC -shared -o c_functions.so c_functions.c

In [3]:
p =  os.path.abspath('.')
lib = ctypes.cdll.LoadLibrary(os.path.join(p, "c_functions.so"))

In [4]:
c_rolling = lib.c_rolling
c_rolling.restype = None
c_rolling.argtypes = [ndpointer(ctypes.c_double, flags="C_CONTIGUOUS"), ctypes.c_int, ctypes.c_int, ndpointer(ctypes.c_double, flags="C_CONTIGUOUS")]

In [5]:
def rolling_average(series: pd.Series, window_length: int=3):
    indata = series.to_numpy(dtype=np.float64)
    nrows = series.shape[0]
    outdata = np.zeros([nrows], dtype=np.float64)
    c_rolling(indata, nrows, window_length, outdata)
    outdata[:(window_length - 1)] = np.NaN
    return pd.Series(outdata)

In [6]:
series = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [7]:
rolling_average(series, 3)

0    NaN
1    NaN
2    2.0
3    3.0
4    4.0
5    5.0
6    6.0
7    7.0
8    8.0
9    9.0
dtype: float64

In [8]:
rolling_average(series, 4)

0    NaN
1    NaN
2    NaN
3    2.5
4    3.5
5    4.5
6    5.5
7    6.5
8    7.5
9    8.5
dtype: float64